# Analysing Borrowers' Risk of Defaulting

# Contents <a id='back'></a>
* [1. Introduction](#intro)
* [2. Data Exploration](#data_exploration)
* [3. Data Preprocessing](#data_preprocessing)
    * [Finding and Analysing Missing Values](#find_missing_values)
    * [Data_Transformation](#data_transformation)
    * [Working_with_Missing_Values](#work_missing_values)
    * [Data Categorisation](#data_categorisation)
* [4. Hypothesis Testing](#hypothesis_testing)
* [5. General Conclusion](#general_conclusion)

## 1. Introduction <a id='intro'></a>
This project is to prepare a report for a bank's loan division. This report will analyse the impact of customer's marital status, number of children, total income, purpose of loan on default rate. The bank has provided some data on customers' credit worthiness to perform the analysis.

This report will be considered when building the credit score of a potential customer. The credit score is used to evaluate the ability of a potential borrower to repay the loan.

### Purpose
This project aims to answer the following questions:
1. Is there a correlation between having children and paying back on time?
2. Is there a correlation between family status and paying back on time?
3. Is there a correlation between income level and paying back on time?
4. How does loan purpose affect the default rate?

### Hypothesis
To answer the questions above, the following hypotheses are formulated:
1. Customers who have no children have a lower default rate compared to those have children.
2. Customers who are married have a lower default rate compared to those are unmarried
3. Customers with high income have the lowest default rate.
4. Customers who borrow a loan for education have the highest default rate.

[Back to Contents](#back)

## 2. Data Exploration <a id='data_exploration'></a>
Open the data provided by the bank.

In [1]:
# Import libraries
import pandas as pd

In [2]:
# Loading data
try:
    df = pd.read_csv('credit_scoring_eng.csv')
except:
    df = pd.read_csv('/datasets/credit_scoring_eng.csv')    

In [3]:
# Check how many columns and rows
df.shape

(21525, 12)

There are 21525 customers and 12 columns of information.

According to the documentation:

**Description of the data**
- `children` - the number of children in the family
- `days_employed` - work experience in days
- `dob_years` - client's age in years
- `education` - client's education
- `education_id` - education identifier
- `family_status` - marital status
- `family_status_id` - marital status identifier
- `gender` - gender of the client
- `income_type` - type of employment
- `debt` - was there any debt on loan repayment
- `total_income` - monthly income
- `purpose` - the purpose of obtaining a loan

In [4]:
# Look at the first 10 rows
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
5,0,-926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house
6,0,-2879.202052,43,bachelor's degree,0,married,0,F,business,0,38484.156,housing transactions
7,0,-152.779569,50,SECONDARY EDUCATION,1,married,0,M,employee,0,21731.829,education
8,2,-6929.865299,35,BACHELOR'S DEGREE,0,civil partnership,1,F,employee,0,15337.093,having a wedding
9,0,-2188.756445,41,secondary education,1,married,0,M,employee,0,23108.150,purchase of the house for my family


In [5]:
# Look at the statictics of the numerical columns
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


**Conclusion:**

From the overview of the data, the dataset has the `children`, `family_status`, `total_income`, `purpose` and `debt` columns which will be used to test the hypotheses. Other columns are good to provide extra information in analysis. However, there are 

**Issues need to be addressed later**
- `children` has negative values
- `days_employed` has negative values and extremely large values
- `dob_years` has '0' value
- `education` is not consistent with the upper case and lower case
- `purpose` is not consistent in wording

[Back to Contents](#back)

## 3. Data Preprocessing <a id='data_preprocessing'></a>

In [6]:
# Get information on data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Both the `days_employed` and `total_income` columns have same amount of missing values. This indicates that they are probably 'missing together'. We will proceed to investigate these missing values.

### Finding and Analysing Missing Values <a id='find_missing_values'></a>

In [7]:
# look through a filtered table containing all the rows with missing values
missing_values_df = df[df.isna().any(axis = 1)]
missing_values_df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


From the table of missing values, it can be seen that
- The total number of rows of missing values is 2174. 
- The missing values in `days_employed` and `total_income` columns seem symmetric. 

To confirm these two types of missing values come from the common rows, we will verify whether the total number of rows which have both  types of missing values is also 2174.

In [8]:
# Find the number of rows with both types of missing values
df[(df['days_employed'].isna()) & (df['total_income'].isna())]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


**Intermediate Conclusion**

The number of rows which have both types of missing value is found to be 2174. It is confirmed that the missing values from `days_employed` and `total_income` indeed coexist in the rows.

In [9]:
# Calculate the percentage of missing values
print(f'Percentage of missing values: {len(missing_values_df) / len(df):.2%}')

Percentage of missing values: 10.10%


Those 10% of missing values will have a huge impact on the outcome of the analysis in later stage. Therefore, we have to fill in these missing values if possible.

Before filling the missing data, it is important to check if the missingness is at random or depend on any customer characteristic.

**Possible reasons for missing values in data**

Since the customers who do not disclose their `total_income` also do not provide the `days_employed` info, this leads us to think that they are not willing to share info about their occupation, so we will check if the missingess is dependent on the `income_type` characteristic.

Besides that, other characteristics could also lead to this missingness. So a function will be created to facilitate this checking process.

In [10]:
# Create a function to check if the missingness is depend on a characteristic (column)

def missing_values_percentage(column):
    '''
    This function calculate the distribution of missing values of total income in a suspected column
    It returns the percentage of missing values for each category
    '''
    
    group = df.groupby(column)
    category_total = group['total_income'].size()
    category_missing_total = group['total_income'].size() - group['total_income'].count()
    category_missing_percentage = (category_missing_total / category_total).apply('{:.2%}'.format)
        
    group_df = pd.concat(
        [category_total, category_missing_total, category_missing_percentage], axis = 1,
        keys = ['total','missing values in total income','% of missing value']
    ).reset_index().sort_values('total',ascending = False)
    
    return group_df

In [11]:
# For income type column, check the distribution of missing values in total income
missing_values_percentage(['income_type'])

,income_type,total,missing values in total income,% of missing value
2,employee,11119,1105,9.94%
0,business,5085,508,9.99%
5,retiree,3856,413,10.71%
1,civil servant,1459,147,10.08%
3,entrepreneur,2,1,50.00%
7,unemployed,2,0,0.00%
4,paternity / maternity leave,1,0,0.00%
6,student,1,0,0.00%


**Intermediate conclusion**

In the `income_type` column, the percentages of missing values in the top four categories having the most customers are about 10%. This is the same as the overall missing values' percentage in whe whole dataset. Therefore, the missingness does not depend on `income_type`.

Although `income_type` is not the reason that causes the missingness, we still cannot conclude that the missingness has no pattern. Since the customers of high education may have more knowledge in protecting personal information, the `education` column will be the next characteristic to be investigated. All the texts will be converted to lower case first before applying the function to check the distribution of the missing values

In [12]:
# For education column, check the distribution of missing values in total income
df['education'] = df['education'].str.lower()
missing_values_percentage(['education'])

,education,total,missing values in total income,% of missing value
3,secondary education,15233,1540,10.11%
0,bachelor's degree,5260,544,10.34%
4,some college,744,69,9.27%
2,primary education,282,21,7.45%
1,graduate degree,6,0,0.00%


**Intermediate conclusion**

In the `education` column, the percentages of missing values in the top two categories having the most customers are about 10%, which is same as the overall missing values' percentage in whe whole dataset. Therefore, the missingness also does not depend on `education`.

To further ensure that the values are missing randomly, we will continue to apply the function to the remaining columns

In [13]:
# For gender column, check the distribution of missing values in total income
missing_values_percentage(['gender'])

,gender,total,missing values in total income,% of missing value
0,F,14236,1484,10.42%
1,M,7288,690,9.47%
2,XNA,1,0,0.00%


In [14]:
# For family_status column, check the distribution of missing values in total income
missing_values_percentage(['family_status'])

,family_status,total,missing values in total income,% of missing value
2,married,12380,1237,9.99%
0,civil partnership,4177,442,10.58%
3,unmarried,2813,288,10.24%
1,divorced,1195,112,9.37%
4,widow / widower,960,95,9.90%


In [15]:
# For children column, check the distribution of missing values in total income
missing_values_percentage(['children'])

,children,total,missing values in total income,% of missing value
1,0,14149,1439,10.17%
2,1,4818,475,9.86%
3,2,2055,204,9.93%
4,3,330,36,10.91%
7,20,76,9,11.84%
0,-1,47,3,6.38%
5,4,41,7,17.07%
6,5,9,1,11.11%


**Intermdiate conclusion**

In the `gender`, `family_status`,`children` columns, almost all the categories have missing values of 10%. The 'XNA' category in `gender` and '4' children category in `children` have some different results but the sample size is too small to say that the missingness is related to them. Thus, the missingness does not depend on these three characteristics too.

**Conclusion**

Since all of the categories with major counts of customers have a missing value's percentage of about 10%, which is same as the overall missing values' percentage, we can conclude that the missingess has no pattern. 

To fill in the missing values in `total_income`, we will identify which characteristic affects `total_income` the most, same strategy goes to the `days_employed`. Before that, we need to addressed on the problematic issues in each column and do some data transformation.

[Back to Contents](#back)

### Data Transformation <a id='data_transformation'></a>

#### Duplicates
We will drop the duplicates first before checking the columns separately.

In [16]:
# Check how many duplicates are there
df.duplicated().sum()

71

In [17]:
# Drop the duplicates
df = df.drop_duplicates().reset_index(drop = True)

In [18]:
# Check if all the duplicates are dropped
df.duplicated().sum()

0

#### `children` column

In [19]:
# Check the distribution of values in the `children` column
df['children'].value_counts().sort_index()

-1        47
 0     14091
 1      4808
 2      2052
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

It is impossible to have '-1' children and '20' children is also highly unlikely. Now we will find the percentage of these values and decide what to do.

In [20]:
# Find the percentage of each value in the 'children' column
(df['children'].value_counts() / len(df)).apply('{:.2%}'.format)

 0     65.68%
 1     22.41%
 2      9.56%
 3      1.54%
 20     0.35%
-1      0.22%
 4      0.19%
 5      0.04%
Name: children, dtype: object

- The value of '-1' may indicate that the customer does not have children, so the value will be reassigned to '0'.
- The value of '20' could be a typo since '2' is near to '0' at the numpad, so the value will be reassigned to '2'.

In [21]:
# Replace '-1' and '20' with '0'
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 2)

# Check if the replacement works
df['children'].value_counts()

0    14138
1     4808
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

#### `days_employed` column

In [22]:
# Look for the problematic issues in 'days_employed'
df['days_employed']

0         -8437.673028
1         -4024.803754
2         -5623.422610
3         -4124.747207
4        340266.072047
             ...      
21449     -4529.316663
21450    343937.404131
21451     -2113.346888
21452     -3112.481705
21453     -1984.507589
Name: days_employed, Length: 21454, dtype: float64

Both the negative values and the extreme large values are impossible to have in `days_employed`. Besides that, the value should be integers to represent number of days. We will check the percentages of these problematic values.

Note: Assume that the maximum employed period is 70 years, which is 70 x 365 = 25500 days, therefore any value which is larger than 25500 which be deemed as abnormal.

In [23]:
# Create tables for the negative and the abnormal large values
negative_days_employed = df[df['days_employed'] < 0]
large_days_employed = df[df['days_employed'] > 25500]

# Count the percetage of the negative values
print(
    'The percentage of the negative values in days_employed: ' + 
    f'{negative_days_employed["days_employed"].count() / df["days_employed"].count():.2%}'
)

# Count the percetage of the extreme large values
print(
    'The percentage of the extreme large values in days_employed: ' + 
    f'{large_days_employed["days_employed"].count() / df["days_employed"].count():.2%}'
)

The percentage of the negative values in days_employed: 82.20%
The percentage of the extreme large values in days_employed: 17.80%


These two percentages add up to 100%, which contributes to all the available data in `days_employed`. Let's look at the statictics to gain more insight of these two groups of problematic data.

In [24]:
# Look at the statistics of days_employed
df['days_employed'].describe()

count     19351.000000
mean      63046.497661
std      140827.311974
min      -18388.949901
25%       -2747.423625
50%       -1203.369529
75%        -291.095954
max      401755.400475
Name: days_employed, dtype: float64

- All the negative values lie between 0 and -18388, since 18388 days is a reasonable employed period, these problematic data should be a mistake in the subtraction calculation when two numbers are reversed, we will modify these values by taking absolute values on them.
- If we divide the maximum by the mimimum, 401755 / 18388 = 21.8 approximately, which is analogue to 24 hours per day. Thus, we may get the correct values by dividing these large numbers by 24.

Before applying these changes, we will check if the modification is reasonble by using `dob_years` minus `days_employed` / 365 to find the age when the customers start working. In the previous data overview section, we notice that the dataset has '0' in `dob_years` which is not reasonable, so we will drop these values first.

In [25]:
# Drop age of 0 in negative_days_employed table
negative_days_employed = negative_days_employed[negative_days_employed['dob_years'] != 0]

# Check if the modification by taking absolute value is reasonable
negative_days_employed['days_employed'] = abs(negative_days_employed['days_employed'])
(negative_days_employed['dob_years'] - (negative_days_employed['days_employed'] / 365)).describe()

count    15832.000000
mean        33.555785
std         10.206013
min         10.619315
25%         25.328778
50%         32.195891
75%         40.705756
max         70.400083
dtype: float64

After taking absolute value on the negative values, the statistics shows that the customers started working at the minimum age of 10.6 and the maximum age of 70.4. These figures are reasonable and we will proceed to make this modification in the original table.

In [26]:
# Take the absolute value of the values in days_employed
df['days_employed'] = abs(df['days_employed'])

# Check if there is any negative value remains
df['days_employed'].describe()

count     19351.000000
mean      66914.728907
std      139030.880527
min          24.141633
25%         927.009265
50%        2194.220567
75%        5537.882441
max      401755.400475
Name: days_employed, dtype: float64

Looks good, next we will deal with the unusual large values in `days_employed`.

In [27]:
# Drop age of 0 in large_days_employed table
large_days_employed = large_days_employed[large_days_employed['dob_years'] != 0]

# Check if the modification by dividing 24 is reasonable
(large_days_employed['dob_years'] - (large_days_employed['days_employed'] / 24 / 365)).describe()

count    3428.000000
mean       17.749473
std         6.744515
min       -18.451731
25%        13.868188
50%        18.026004
75%        22.263458
max        35.012771
dtype: float64

Looks like this modification is not reasonable since there is a customer started working at -18.5 years old. We will look into the table to see if we can find any pattern.

In [28]:
# Check the large_days_employed table
large_days_employed

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding
18,0,400281.136913,53,secondary education,1,widow / widower,2,F,retiree,0,9091.804,buying a second-hand car
24,1,338551.952911,57,secondary education,1,unmarried,4,F,retiree,0,46487.558,transactions with commercial real estate
25,0,363548.489348,67,secondary education,1,married,0,M,retiree,0,8818.041,buy real estate
30,1,335581.668515,62,secondary education,1,married,0,F,retiree,0,27432.971,transactions with commercial real estate
...,...,...,...,...,...,...,...,...,...,...,...,...
21434,0,338904.866406,53,secondary education,1,civil partnership,1,M,retiree,0,12070.399,to have a wedding
21437,0,386497.714078,62,secondary education,1,married,0,M,retiree,0,11622.175,property
21438,0,362161.054124,59,bachelor's degree,0,married,0,M,retiree,0,11684.650,real estate transactions
21447,0,373995.710838,59,secondary education,1,married,0,F,retiree,0,24618.344,purchase of a car


It seems like all the rows have retiree as the `income_type`. Let's check on this.

In [29]:
# Check if all the customers are retiree in large_days_employed
large_days_employed.groupby('income_type')['days_employed'].count()

income_type
retiree       3426
unemployed       2
Name: days_employed, dtype: int64

All these large numbers of `days_employed` come from the retiree `income_type` with only 2 exceptions. SInce the majority of retirees should be older people, we will check their age distribution.

In [30]:
# Check the age distribution in large_days_employed
large_days_employed['dob_years'].value_counts().sort_index(ascending = False)

74      4
73      6
72     28
71     48
70     54
69     74
68     80
67    132
66    139
65    136
64    179
63    192
62    235
61    214
60    243
59    254
58    208
57    212
56    184
55    162
54    145
53    105
52     95
51     73
50     61
49     30
48     20
47     13
46     13
45     11
44     10
43      9
42      9
41      6
40      7
39      4
38      8
37      5
36      3
35      1
34      3
33      2
32      3
31      1
28      1
27      3
26      2
22      1
Name: dob_years, dtype: int64

In [31]:
# Check the percentage of retiree having age larger or equal to 50
print(
    'Percentage of retirees who are 50 years old and above: ' +
    f'{len(large_days_employed[large_days_employed["dob_years"] >= 50]) / len(large_days_employed):.2%}'
)

Percentage of retirees who are 50 years old and above: 95.19%


There are about 95% of the retirees aged 50 and above. To modify the large `days_employed` values, we will use the `days_employed` data from the negative_days_employed group and have a check on the mean and the median first.

In [32]:
# Check the required mean and the median
negative_days_employed[negative_days_employed['dob_years'] >= 50]['days_employed'].agg(['mean','median'])

mean      3352.860839
median    2323.563512
Name: days_employed, dtype: float64

The median is less than the mean by about 1000, this shows that the distribution is positively skewed and we will take the median to replace the abnormal values in `days_employed`.

In [33]:
# Set up the median
days_employed_median = negative_days_employed[negative_days_employed['dob_years'] >= 50]['days_employed'].median()

# Check if the replacement is reasonable
large_days_employed.loc[:,'days_employed'] = days_employed_median

(large_days_employed['dob_years'] - (large_days_employed['days_employed'] / 365)).describe()

count    3428.000000
mean       53.052101
std         6.350063
min        15.634073
25%        49.634073
50%        53.634073
75%        57.634073
max        67.634073
dtype: float64

The statistics shows that this replacement with median is reasonable since customers started working at the minimum age of 15.6 and the maximum age of 67.6. Now the change will be applied to the original table.

In [34]:
# Replace the abnormal values in days_employed
df.loc[df['days_employed'] > 25500, 'days_employed'] = days_employed_median

# Final check at the days_employed column
df['days_employed'].describe()

count    19351.000000
mean      2347.772607
std       2089.109025
min         24.141633
25%        927.009265
50%       2194.220567
75%       2747.423625
max      18388.949901
Name: days_employed, dtype: float64

Everything is fixed. Now we will continue with the `dob_years` column.

#### `dob_years` column

In [35]:
# Check the values in dob_years
df['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

In [36]:
# Check the percentage of 0 years old customers
print(
    'The percentage of 0 years old customers:' +
    f'{len(df[df["dob_years"] == 0]) / len(df):.2%}'
)

The percentage of 0 years old customers:0.47%


A value of '0' in this column is impossible since a '0' years old baby is incapble of borrowing a loan. The possible reason may be the customer does not want to reveal his/her age. We cannot replace '0' with mean or median of `dob_years` because it will substantially increase the count of that age. Since this error only accounts for 0.47% for the whole dataset and `dob_years` is not used in the hypothesis testing, we will replace it with 'nan' to keep other valuable information.

In [37]:
# Replace '0' with 'nan' in dob_years
df['dob_years'] = df['dob_years'].replace(0, float('nan'))

# Final check on the values in dob_years
df['dob_years'].sort_values().unique()

array([19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31.,
       32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44.,
       45., 46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57.,
       58., 59., 60., 61., 62., 63., 64., 65., 66., 67., 68., 69., 70.,
       71., 72., 73., 74., 75., nan])

Everything is fixed.

#### `family_status` column

In [38]:
# Look at the values in family_status column
df['family_status'].value_counts()

married              12339
civil partnership     4151
unmarried             2810
divorced              1195
widow / widower        959
Name: family_status, dtype: int64

Everything looks good in the `family_status` column

#### `gender` column

In [39]:
# Let's see the values in the column
df['gender'].value_counts()

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

There is a customer with gender 'XNA', this can be a third gender or the customer does not want to reveal this information. We cannot confirm which is the case and we will replace it with 'unknown'.

In [40]:
# Replace the XNA in gender with unknown
df.loc[df['gender'] == 'XNA', 'gender'] = 'unknown'

# Check if the replacement is successful
df['gender'].value_counts()

F          14174
M           7279
unknown        1
Name: gender, dtype: int64

#### `income_type` column

In [41]:
# Check the value in income_type column
df['income_type'].value_counts()

employee                       11084
business                        5078
retiree                         3829
civil servant                   1457
unemployed                         2
entrepreneur                       2
student                            1
paternity / maternity leave        1
Name: income_type, dtype: int64

Everything seems fine in `income_type` column. 

We have done the checking task on every column except the `purpose` column. The `purpose` column will be categorised in the later stage to investigate the hypothesis. Now have a last check on the size of the dataset after the first round of the manipulation.

In [42]:
# Check the size of the dataset and compared to that of the original dataset
print(f'The dataset now has {len(df)} rows')
print(f'The remaining rows of the current dataset is {len(df) / 21525:.2%} of the original dataset')

The dataset now has 21454 rows
The remaining rows of the current dataset is 99.67% of the original dataset


We are good to deal with the missing values now.

[Back to Contents](#back)

### Working with Missing Values <a id='work_missing_values'></a>

To speed up our works, two dictionaries of data with provided IDs will be identified and used in the later sections.

In [43]:
# Find the dictionary with education id and education
education_dict_df = df[['education_id','education']].drop_duplicates().reset_index(drop = True)
education_dict_df

,education_id,education
0,0,bachelor's degree
1,1,secondary education
2,2,some college
3,3,primary education
4,4,graduate degree


In [44]:
# Find the dictionary with family status id and family status
family_status_dict_df = df[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop = True)
family_status_dict_df

,family_status_id,family_status
0,0,married
1,1,civil partnership
2,2,widow / widower
3,3,divorced
4,4,unmarried


Here are the dictionaries for `education` and `family_status`, now we can use their corresponding IDs to speed up some calculation works.

Next, we will address on the missing values in `total_income` and `days_employed` which have been identified earlier.

#### Restoring missing values in`total_income` column
To fill the missing values in `total_income`, we will find the deciding factors (columns) that affect `total_income` and fill in the missing values with the mean (or median, depends on which is better) of these factors. 

The deciding factors will be found by comparing the means and medians of `total_income` from different characteristics (column). If the means and medians of the categories in a specific characteristic vary by a large scale, then the characteristic will be the deciding factor.

Before that, we have to group the `dob_years` into several age groups so that we can use them to compare the means and medians.

In [45]:
# Define a function which returns the age group
def assign_age_group(age):
    if age < 0 or pd.isna(age):
        return 'NA'
    elif age < 10:
        return '0-9'
    elif age < 20:
        return '10-19'
    elif age < 30:
        return '20-29'
    elif age < 40:
        return '30-39'
    elif age < 50:
        return '40-49'
    elif age < 60:
        return '50-59'
    elif age < 70:
        return '60-69'
    else:
        return '70+'

In [46]:
# Test if the function works
print(assign_age_group(float('nan')))
print(assign_age_group(5))
print(assign_age_group(44))
print(assign_age_group(70))

NA
0-9
40-49
70+


In [47]:
# Create a new column age_group and apply the function
df['age_group'] = df['dob_years'].apply(assign_age_group)

# Check how does the new column look
df[['dob_years', 'age_group']].head(10)

,dob_years,age_group
0,42.0,40-49
1,36.0,30-39
2,33.0,30-39
3,32.0,30-39
4,53.0,50-59
5,27.0,20-29
6,43.0,40-49
7,50.0,50-59
8,35.0,30-39
9,41.0,40-49


In [48]:
# Check the distribution of the age_group
df['age_group'].value_counts()

30-39    5662
40-49    5354
50-59    4657
20-29    3166
60-69    2331
70+       169
NA        101
10-19      14
Name: age_group, dtype: int64

We are good to start finding the deciding factors. As we have some missing values in `dob_years` and `age_group`, we have to drop them first so that the means and medians will not be influenced by the uncertainty of these missing values and have a better accuracy.

In [49]:
# Create a table without missing values
df_without_na = df.dropna().reset_index(drop = True)

# Look at the table without missing values
df_without_na

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,8437.673028,42.0,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36.0,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33.0,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32.0,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
4,0,2323.563512,53.0,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19255,1,4529.316663,43.0,secondary education,1,civil partnership,1,F,business,0,35966.698,housing transactions,40-49
19256,0,2323.563512,67.0,secondary education,1,married,0,F,retiree,0,24959.969,purchase of a car,60-69
19257,1,2113.346888,38.0,secondary education,1,civil partnership,1,M,employee,1,14347.610,property,30-39
19258,3,3112.481705,38.0,secondary education,1,married,0,M,employee,1,39054.888,buying my own car,30-39


The `total_income` is likely to be decided by the occupation, therefore the first characteristic to be investigated is the `income_type` column.

In [50]:
# Check the means and medians of total_income for each category in income_type
df_without_na.groupby('income_type')['total_income'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
income_type,,,
employee,9964,25824.679592,22815.1035
business,4559,32397.357125,27564.8930
retiree,3426,21939.310393,18969.1490
civil servant,1306,27361.316126,24083.5065
unemployed,2,21014.360500,21014.3605
entrepreneur,1,79866.103000,79866.1030
paternity / maternity leave,1,8612.661000,8612.6610
student,1,15712.260000,15712.2600


As excepected, the means and medians in the categories with most counts of customers vary substantially. Hence, `income_type` will be one of the deciding factors to be used. 

Usually, the higher the education, the higher will be the earning, thus `education` will be the second characteristic to be considered.

In [51]:
# Check the means and medians of total_income for each category in education
df_without_na.groupby('education')['total_income'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
education,,,
secondary education,13636,24600.353617,21839.4075
bachelor's degree,4684,33172.428387,28054.5310
some college,673,29040.391842,25618.4640
primary education,261,21144.882211,18741.9760
graduate degree,6,27960.024667,25161.5835


Since the means and medians also vary greatly across the categories, `education` will be used to fill in the missing values in `total_income`.

It is suspected that the `age_group` is relevant to the `total_income` as elder people should have more work experience and may get higher salaries. Let's have a check.

In [52]:
# Check the means and medians of total_income for each category in age_group
df_without_na.groupby('age_group')['total_income'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
age_group,,,
30-39,5109,28312.479963,24667.5280
40-49,4834,28551.375635,24764.2290
50-59,4178,25811.700327,22203.0745
20-29,2871,25572.630177,22799.2580
60-69,2095,23242.812818,19817.4400
70+,160,20125.658331,18751.3240
10-19,13,16993.942462,14934.9010


The means and the medians with the top five counts of customers vary in a range of 5000. Since the differences between categories are not significant compared to the `income_type` and `education`, `age_group` will not be considered as a deciding factor. We will continue to check on the remaining three categories, namely `children`, `family_status` and `gender`.

In [53]:
# Check the means and medians of total_income for each category in children
df_without_na.groupby('children')['total_income'].agg(['count','mean','median'])

,count,mean,median
children,,,
0,12693,26423.791819,23034.0880
1,4328,27405.559686,23661.4030
2,1904,27469.471068,23127.7930
3,293,29366.910652,25191.6190
4,34,27289.829647,24981.6340
5,8,27268.847250,29816.2255


In [54]:
# Check the means and medians of total_income for each category in family_status
df_without_na.groupby('family_status')['total_income'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
family_status,,,
married,11098,27045.383530,23377.7080
civil partnership,3717,26703.842697,23199.2980
unmarried,2510,26943.601742,23139.4040
divorced,1074,27202.683563,23584.9695
widow / widower,861,23006.808776,20523.2670


In [55]:
# Check the means and medians of total_income for each category in gender
df_without_na.groupby('gender')['total_income'].agg(['count','mean','median'])

,count,mean,median
gender,,,
F,12688,24664.752169,21469.0015
M,6571,30905.772981,26819.5670
unknown,1,32624.825000,32624.8250


The means and medians in the `children` and `family_status` characteristics does not vary widely and will be excluded as the factors to evaluate missing values. Surprisingly, the means and medians for male and female in the `gender` characteristic differ considerably between the categories, therefore gender will be taken as one of the factors.

As a result, `income_type`, `age_group` and `gender` will be the deciding factors to fill in the missing values of `total_income`. Since all the medians are lower than the means, the distribution is positively skewed and thus median is preferred over mean to fill in the missing values.

Now, we will write a function to fill in these missing values. The function will find the missing values and fill them with the overall median of the corresponding factors. Here we will use the `education_id` to speed up the calculation.

In [56]:
# Write the function to fill in the missing values of total_income
def fill_income_missing_values(row):
    
    # Create a variable to store the income value for the row
    income = row['total_income']
    
    # If the row has a missing value in total_income
    # Check the income type, education id and gender of the row
    # And find the corresponding income median
    if pd.isna(row['total_income']):
        income = df_without_na[
            (df_without_na['income_type'] == row['income_type']) & 
            (df_without_na['education_id'] == row['education_id']) &
            (df_without_na['gender'] == row['gender'])
        ]['total_income'].median()
        
    return income

In [57]:
# Find a row with missing values to check if the functions work
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
12,0,NaN,65.0,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding,60-69
26,0,NaN,41.0,secondary education,1,married,0,M,civil servant,0,NaN,education,40-49
29,0,NaN,63.0,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate,60-69
41,0,NaN,50.0,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase,50-59
55,0,NaN,54.0,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21418,2,NaN,47.0,secondary education,1,married,0,M,business,0,NaN,purchase of a car,40-49
21424,1,NaN,50.0,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony,50-59
21426,0,NaN,48.0,bachelor's degree,0,married,0,F,business,0,NaN,building a property,40-49
21431,1,NaN,42.0,secondary education,1,married,0,F,employee,0,NaN,building a real estate,40-49


In [58]:
# Manually find the median for the missing value in the row with index 12
df_without_na[
    (df_without_na['income_type'] == df.loc[12, 'income_type']) & 
    (df_without_na['education_id'] == df.loc[12, 'education_id']) &
    (df_without_na['gender'] == df.loc[12, 'gender']) 
        ]['total_income'].median()

20180.8795

In [59]:
# Check if the function gives the same median
fill_income_missing_values(df.loc[12,:])

20180.8795

In [60]:
# Apply the function to every row
df['total_income'] = df.apply(fill_income_missing_values, axis = 1)

In [61]:
# Check if all the missing values are filled in
df['total_income'].isna().sum()

1

Now all the missing values are filled in with one exception. We need to check on this row and try to fixed it manually.

In [62]:
# Look at the only row with a missing value in total income
df[df['total_income'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5931,0,NaN,58.0,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59


The row of index 5931 has an `income_type` of entrepreneur which is a minority in the group. Let's check on this.

In [63]:
df[df['income_type'] == 'entrepreneur']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
5931,0,NaN,58.0,bachelor's degree,0,married,0,M,entrepreneur,0,NaN,buy residential real estate,50-59
18647,0,520.848083,27.0,bachelor's degree,0,civil partnership,1,F,entrepreneur,0,79866.103,having a wedding,20-29


There are only 2 customers with the `income_type` of entrepreneur. To fill up this missing values, we will remove `income_type` as a deciding factor and use the remaining two factors to find the median.

In [64]:
# Use education_id and gender to find median
total_income_5931 = df[(df['education_id'] == 0) & (df['gender'] == 'M')]['total_income'].median()

# Fill in the last missing value in total income
df['total_income'] = df['total_income'].fillna(total_income_5931)

In [65]:
# Check if all the missing values in total income are filled in
print('Number of values in total_income: ' + str(df['total_income'].count()))
print('Total number of rows: ' + str(len(df)))

Number of values in total_income: 21454
Total number of rows: 21454


All the missing values in `total_income` are fixed. 

The values in `total_income` should at most have two decimal places. Round() method will be applied on these values to get have two decimal places approximation.

In [66]:
# Round the values in `total_income` to two decimal places
df['total_income'] = df['total_income'].round(2)

# Check if the rounding works
df['total_income']

0        40620.10
1        17932.80
2        23341.75
3        42820.57
4        25378.57
           ...   
21449    35966.70
21450    24959.97
21451    14347.61
21452    39054.89
21453    13127.59
Name: total_income, Length: 21454, dtype: float64

Grest, we will continue with the `days_employed` columns with a similar method.

#### Restoring missing values in`days_employed` column
Since `days_employed` is directly related to age, we will first examine the means and medians in the `age_group`

In [67]:
# Check the means and medians of days_employed for each category in age_group
df_without_na.groupby('age_group')['days_employed'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
age_group,,,
30-39,5109,2028.156331,1601.784231
40-49,4834,2722.379699,2111.489906
50-59,4178,2924.557667,2323.563512
20-29,2871,1215.252787,1005.629955
60-69,2095,2664.305297,2323.563512
70+,160,2561.469189,2323.563512
10-19,13,633.678086,724.492610


The means and medians alters significantly across the top five categories with the highest customer counts. The `age_group` indeed is a deciding factor which will affect `days_employed` values.

It is possible that the `days_employed` will be lower if customers have children because they have to spend time to take care of their children. Therefore, we will check the `children` column next.

In [68]:
df_without_na.groupby('children')['days_employed'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
children,,,
0,12693,2444.925476,2323.563512
1,4328,2173.983922,1666.838773
2,1904,2133.664760,1681.743105
3,293,2180.070236,1734.879824
4,34,2184.140337,1905.879025
5,8,1432.348601,1231.571486


Generally, the medians and means are almost the same except the customers without children have slightly higher `days_employed` values. This is not sufficient for us to take in `children` as a factor to fill in the missing values. 

It is suspected that a married customer will work harder to support the family, so we will look into the `family_status` to see if it has any impact on `days_employed`.

In [69]:
# Check the means and medians of total_income for each category in family_status
df_without_na.groupby('family_status')['days_employed'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
family_status,,,
married,11098,2446.404251,2306.974767
civil partnership,3717,2265.787696,1948.471496
unmarried,2510,1906.279932,1460.777023
divorced,1074,2419.414636,2323.563512
widow / widower,861,2641.205735,2323.563512


The top three categories show huge differences in means and medians. As a result, `family_status` will be taken as one of the deciding factors.

We will check the rest three columns, namely `income_type', 'education` and `gender` to make sure we do not left out any possible factor.

In [70]:
# Check the means and medians of total_income for each category in income_type
df_without_na.groupby('income_type')['days_employed'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
income_type,,,
employee,9964,2328.603723,1576.067689
business,4559,2112.798330,1548.637544
retiree,3426,2323.563512,2323.563512
civil servant,1306,3388.508552,2673.404956
unemployed,2,2323.563512,2323.563512
entrepreneur,1,520.848083,520.848083
paternity / maternity leave,1,3296.759962,3296.759962
student,1,578.751554,578.751554


In [71]:
# Check the means and medians of total_income for each category in education
df_without_na.groupby('education')['days_employed'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
education,,,
secondary education,13636,2412.370327,2323.563512
bachelor's degree,4684,2282.718500,1895.747795
some college,673,1606.585075,1209.230373
primary education,261,2077.105224,2323.563512
graduate degree,6,3121.312195,2337.497723


In [72]:
# Check the means and medians of total_income for each category in gender
df_without_na.groupby('gender')['days_employed'].agg(['count','mean','median']).sort_values('count', ascending = False)

,count,mean,median
gender,,,
F,12688,2476.071083,2323.563512
M,6571,2101.760893,1667.140839
unknown,1,2358.600502,2358.600502


Out of the three categories, only the medians and means in `income_type` vary considerably, especally in the civil servant category. 

With the above findings, we will use `age_group', 'family_status_id` and `income_type` as the factors to fill in the missing values in `days_employed`. Similarly, median will be used since the distribution is skewed.

In [73]:
# Write the function to fill in the missing values of days_employed
def fill_days_employed_missing_values(row):
    
    # Create a variable to store the days_employed value for the row
    days_employed = row['days_employed']
    
    # If the row has a missing value in days_employed
    # Check the age_group, family_status_id and income_type of the row
    # And find the corresponding income median
    if pd.isna(row['days_employed']):
        days_employed = df_without_na[
            (df_without_na['age_group'] == row['age_group']) & 
            (df_without_na['family_status_id'] == row['family_status_id']) &
            (df_without_na['income_type'] == row['income_type'])
        ]['days_employed'].median()
        
    return days_employed

In [74]:
# Find a row with missing values to check if the functions work
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
12,0,NaN,65.0,secondary education,1,civil partnership,1,M,retiree,0,20180.88,to have a wedding,60-69
26,0,NaN,41.0,secondary education,1,married,0,M,civil servant,0,27144.31,education,40-49
29,0,NaN,63.0,secondary education,1,unmarried,4,F,retiree,0,18046.56,building a real estate,60-69
41,0,NaN,50.0,secondary education,1,married,0,F,civil servant,0,20133.09,second-hand car purchase,50-59
55,0,NaN,54.0,secondary education,1,civil partnership,1,F,retiree,1,18046.56,to have a wedding,50-59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21418,2,NaN,47.0,secondary education,1,married,0,M,business,0,29102.24,purchase of a car,40-49
21424,1,NaN,50.0,secondary education,1,civil partnership,1,F,employee,0,19872.94,wedding ceremony,50-59
21426,0,NaN,48.0,bachelor's degree,0,married,0,F,business,0,30352.88,building a property,40-49
21431,1,NaN,42.0,secondary education,1,married,0,F,employee,0,19872.94,building a real estate,40-49


In [75]:
# Manually find the median for the missing value in the row with index 12
df_without_na[
    (df_without_na['age_group'] == df.loc[12, 'age_group']) & 
    (df_without_na['family_status_id'] == df.loc[12, 'family_status_id']) &
    (df_without_na['income_type'] == df.loc[12, 'income_type'])
]['days_employed'].median()

2323.5635122338854

In [76]:
# CHeck if the function give the same median
fill_days_employed_missing_values(df.loc[12,:])

2323.5635122338854

In [77]:
# Apply the function to every row
df['days_employed'] = df.apply(fill_days_employed_missing_values, axis = 1)

In [78]:
# Check if all missing values are filled in
df['days_employed'].isna().sum()

12

There are 12 missing values cannot be filled by the function. We have to look into the table to find out the reason.

In [79]:
# Check all the rows with missing values in days_employed
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1890,0,NaN,NaN,bachelor's degree,0,unmarried,4,F,employee,0,24592.44,housing,NA
2284,0,NaN,NaN,secondary education,1,widow / widower,2,F,retiree,0,18046.56,property,NA
4050,0,NaN,19.0,secondary education,1,civil partnership,1,M,employee,0,24985.85,having a wedding,10-19
4062,1,NaN,NaN,secondary education,1,civil partnership,1,M,business,0,29102.24,housing renovation,NA
5010,0,NaN,NaN,secondary education,1,married,0,F,business,0,23178.82,buy real estate,NA
5931,0,NaN,58.0,bachelor's degree,0,married,0,M,entrepreneur,0,32521.31,buy residential real estate,50-59
6405,0,NaN,NaN,bachelor's degree,0,civil partnership,1,F,retiree,0,22784.68,wedding ceremony,NA
6664,0,NaN,NaN,bachelor's degree,0,divorced,3,F,retiree,0,22784.68,buy residential real estate,NA
8565,0,NaN,NaN,secondary education,1,married,0,F,employee,0,19872.94,property,NA
12381,3,NaN,NaN,secondary education,1,married,0,M,employee,0,24985.85,transactions with commercial real estate,NA


By analysing the table, it is decided that the missing values will be filled in by three different ways:
- The row with index 4050 is a customer of aged 19. Since the `age_group` of 10-19 has only 13 customers, we will take in all of these 13 rows to find a 'reasonable good' value to fill in this missing value.
- The row with index 5931 is the customer with entrepreneur as `income_type`, we will take this out and keep `age_group` and `family_status` as factors to fill the missing values.
- There are 10 rows without data in `age_group`, so we will use the remaining two factors `family_status` and `income_type` to fill in the missing values.

In [80]:
# Use age_group as the factor for the row with index 4050
days_employed_4050 = df[df['age_group'] == '10-19']['days_employed'].median()
# Fill the missing value
df.loc[4050, 'days_employed'] = days_employed_4050


# Use age_group and family_status_id as factors for the row with index 5931
days_employed_5931 = df[(df['age_group'] == '50-59') & (df['family_status_id'] == 0)]['days_employed'].median()
# Fill the missing value in the row with index 5931
df.loc[5931, 'days_employed'] = days_employed_5931

In [81]:
df[df['days_employed'].isna()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
1890,0,NaN,NaN,bachelor's degree,0,unmarried,4,F,employee,0,24592.44,housing,NA
2284,0,NaN,NaN,secondary education,1,widow / widower,2,F,retiree,0,18046.56,property,NA
4062,1,NaN,NaN,secondary education,1,civil partnership,1,M,business,0,29102.24,housing renovation,NA
5010,0,NaN,NaN,secondary education,1,married,0,F,business,0,23178.82,buy real estate,NA
6405,0,NaN,NaN,bachelor's degree,0,civil partnership,1,F,retiree,0,22784.68,wedding ceremony,NA
6664,0,NaN,NaN,bachelor's degree,0,divorced,3,F,retiree,0,22784.68,buy residential real estate,NA
8565,0,NaN,NaN,secondary education,1,married,0,F,employee,0,19872.94,property,NA
12381,3,NaN,NaN,secondary education,1,married,0,M,employee,0,24985.85,transactions with commercial real estate,NA
13716,0,NaN,NaN,secondary education,1,civil partnership,1,F,employee,0,19872.94,having a wedding,NA
19770,0,NaN,NaN,secondary education,1,married,0,F,employee,0,19872.94,housing,NA


In [82]:
# Use family_status_id and income_type as factors to fill the rest
df['days_employed'] = df['days_employed'].fillna(
    df.groupby(['family_status_id','income_type'])['days_employed'].transform('median')
)

In [83]:
# Last check if all missing values are fixed
df.isna().sum()

children              0
days_employed         0
dob_years           101
education             0
education_id          0
family_status         0
family_status_id      0
gender                0
income_type           0
debt                  0
total_income          0
purpose               0
age_group             0
dtype: int64

In `days_employed` column, the values should be integers to represent number of days. 

We will use the round() method to round off the values in `days_employed` and convert them to integers by astype('int')

In [84]:
# Round the values in `days_employed` to integers
df['days_employed'] = df['days_employed'].round().astype('int32')

# Check the data type of days_employed
df['days_employed']

0        8438
1        4025
2        5623
3        4125
4        2324
         ... 
21449    4529
21450    2324
21451    2113
21452    3112
21453    1985
Name: days_employed, Length: 21454, dtype: int32

The dataset looks good now.

[Back to Contents](#back)

### Data Categorisation <a id='data_categorisation'></a>
Before testing the hypotheses, some data needs to be categorised.
- `total_income` will be categorised since no comaparison can be made with all different values in the column.
- `purpose` will be categorised since many of these purposes are actually similar and we can group them into a few main categories to have a clearer comaparison in hypothesis testing.

#### `total_income`

In [85]:
# Looks at the statistics of total_income
df['total_income'].describe()

count     21454.000000
mean      26475.125169
std       15744.042763
min        3306.760000
25%       17200.597500
50%       23178.820000
75%       31507.715000
max      362496.640000
Name: total_income, dtype: float64

The `total_income` column has 21454 values. By using the three quartiles, these values will be categorised as follows:
- less than 17200 : low
- between 17200 and 23200: lower medium
- between 23200 and 31500: upper medium
- more than 31500: high

In [86]:
# Write a function to assgin income level
def assign_income_level(income):
    if income < 0 or pd.isna(income):
        return 'NA'
    elif income <= 17200:
        return 'low'
    elif income <= 23200:
        return 'lower medium'
    elif income <= 31500:
        return 'upper medium'
    else:
        return 'high'

In [87]:
# Test if the income level function works
print(assign_income_level(10000))
print(assign_income_level(30000))
print(assign_income_level(200000))

low
upper medium
high


In [88]:
# Assign income group in a new column
df['income_level'] = df['total_income'].apply(assign_income_level)

In [89]:
# Check if the group is successfully assigned
df[['total_income','income_level']].head(10)

,total_income,income_level
0,40620.10,high
1,17932.80,lower medium
2,23341.75,upper medium
3,42820.57,high
4,25378.57,upper medium
5,40922.17,high
6,38484.16,high
7,21731.83,lower medium
8,15337.09,low
9,23108.15,lower medium


In [90]:
# Count each income level group
df['income_level'].value_counts()

lower medium    5453
high            5368
low             5364
upper medium    5269
Name: income_level, dtype: int64

#### `purpose`

In [91]:
# Look at the values in purpose
df['purpose'].sort_values().unique()

array(['building a property', 'building a real estate',
       'buy commercial real estate', 'buy real estate',
       'buy residential real estate', 'buying a second-hand car',
       'buying my own car', 'buying property for renting out', 'car',
       'car purchase', 'cars', 'construction of own property',
       'education', 'getting an education', 'getting higher education',
       'going to university', 'having a wedding', 'housing',
       'housing renovation', 'housing transactions', 'profile education',
       'property', 'purchase of a car', 'purchase of my own house',
       'purchase of the house', 'purchase of the house for my family',
       'real estate transactions', 'second-hand car purchase',
       'supplementary education', 'to become educated', 'to buy a car',
       'to get a supplementary education', 'to have a wedding',
       'to own a car', 'transactions with commercial real estate',
       'transactions with my real estate', 'university education',
       'we

From the list of uniques values, these purposes can generally categorised into four groups:
- house/real estate
- car
- education
- wedding

In [92]:
# Write a function to assign loan purpose
def assign_loan_purpose(purpose):
    if 'hous' in purpose or 'real estate' in purpose or 'property' in purpose:
        return 'house/real estate'
    elif 'car' in purpose:
        return 'car'
    elif 'edu' in purpose or 'university' in purpose:
        return 'education'
    elif 'wedding' in purpose:
        return 'wedding'
    else:
        return 'other'

In [93]:
# Test if the loan purpose function works
print(assign_loan_purpose('buy a house'))
print(assign_loan_purpose('further education'))

house/real estate
education


In [94]:
# Assign loan purpose in a new column
df['loan_purpose'] = df['purpose'].apply(assign_loan_purpose)

In [95]:
# Check the unique values to see if all values are categorised
df['loan_purpose'].unique()

array(['house/real estate', 'car', 'education', 'wedding'], dtype=object)

The data preprocessing stage is finished. We will continue to test the hypotheses.

[Back to Contents](#back)

## 4. Hypothesis Testing <a id='hypothesis_testing'></a>

### Hypothesis 1: Customers who have no children have a lower default rate compared to those have children.

In [96]:
# Set up a pivot table with children as index and debt as values
# Calculate the number of customers, number of customers defaulting loans
# And default rates for each category
children_pivot = df.pivot_table(    
    index = 'children',    
    values = 'debt',
    aggfunc = ['count','sum'],    margins = True
)

children_pivot.columns = ['number of customers','loan default']
children_pivot['default rate'] = (
    children_pivot['loan default'] / children_pivot['number of customers']
).apply('{:.2%}'.format)

children_pivot

,number of customers,loan default,default rate
children,,,
0,14138,1064,7.53%
1,4808,444,9.23%
2,2128,202,9.49%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%
All,21454,1741,8.12%


**Conclusion**

The hypothesis is accepted:
- The customers without children have the lowest default rate in the `children` group.

Other findings:
- Ths customers with 1 or 2 children have default rates which are higher than the average rate of 8.12% are more likely to default on their loans.
- The customers with 4 children also have a default rate which is higher than the average, but the sample size is too small to have a proper conclusion.

### Hypothesis 2: Customers who are married have a lower default rate compared to those are unmarried.

In [97]:
# Set up a pivot table with family_status as index and debt as values
# Calculate the number of customers, number of customers defaulting loans
# And default rates for each category
family_status_pivot = df.pivot_table(
    index = ['family_status'],
    values = 'debt',
    aggfunc = ['count','sum'],
    margins = True
)

family_status_pivot.columns = ['number of customers', 'loan default']
family_status_pivot['default rate'] = (
    family_status_pivot['loan default'] / family_status_pivot['number of customers']
).apply('{:.2%}'.format)

family_status_pivot

,number of customers,loan default,default rate
family_status,,,
civil partnership,4151,388,9.35%
divorced,1195,85,7.11%
married,12339,931,7.55%
unmarried,2810,274,9.75%
widow / widower,959,63,6.57%
All,21454,1741,8.12%


**Conclusion**

The hypothesis is accepted:
- The married customers have a lower default rate than the unmarried customers.

Other findings:
- The widow / widower and the divorced categories have the lowest default rates.
- Default rates of the customers with `family_status` of civil partnership and unmarried are higher than the average.

### Hypothesis 3: Customers with high income have the lowest default rate.

In [98]:
# Set up a pivot table with income_level as index and debt as values
# Calculate the number of customers, number of customers defaulting loans
# And default rates for each category
income_level_pivot = df.pivot_table(
    index = 'income_level',
    values = 'debt',
    aggfunc = ['count','sum'],
    margins = True
)

income_level_pivot.columns = ['number of customers', 'loan default']
income_level_pivot['default rate'] = (
    income_level_pivot['loan default'] / income_level_pivot['number of customers']
).apply('{:.2%}'.format)

index_order = ['high', 'upper medium', 'lower medium', 'low', 'All']
income_level_pivot = income_level_pivot.reindex(index_order)

income_level_pivot

,number of customers,loan default,default rate
income_level,,,
high,5368,379,7.06%
upper medium,5269,465,8.83%
lower medium,5453,470,8.62%
low,5364,427,7.96%
All,21454,1741,8.12%


**Conclusion**

Hypothesis is accepted:
- The customers of high income level have the lowest default rate of 7.06%.

Other findings:
- The default rates for the rest of the group are close to the average rate of 8.12%.
- Surprisingly, the default rates of upper medium and lower medium income customers are higher than that of the low income customers.

### Hypothesis 4: Customers who borrow a loan for education have the highest default rate.

In [99]:
# Set up a pivot table with loan_purpose as index and debt as values
# Calculate the number of customers, number of customers defaulting loans
# And default rates for each category
loan_purpose_pivot = df.pivot_table(
    index = 'loan_purpose',
    values = 'debt',
    aggfunc = ['count','sum'],
    margins = True
)

loan_purpose_pivot.columns = ['number of customers','loan default']
loan_purpose_pivot['default rate'] = (
    loan_purpose_pivot['loan default'] / loan_purpose_pivot['number of customers']
).apply('{:.2%}'.format)

loan_purpose_pivot

,number of customers,loan default,default rate
loan_purpose,,,
car,4306,403,9.36%
education,4013,370,9.22%
house/real estate,10811,782,7.23%
wedding,2324,186,8.00%
All,21454,1741,8.12%


**Conclusion**

Hypothesis is rejected:
- The customers who borrow loans for education have the second highest default rate.

Other findings:
- The customers who buy the cars with the loans have the highest default rate.
- The customers who use the loans for house/real estate purpose have the lowest default rate.

[Back to Contents](#back)

## 5. General Conclusion <a id='general_conclusion'></a>
**Data exploration**

The `children`, `family_status`, `total_income`, `purpose` and `debt` data were identified to be used in answering the questions and testing the hypotheses related to default rate. However,there were quite a few problematic issues in the dataset needed to be addressed.

**Data preprocessing**

We found that `days_employed` and `total_income` have same amounts of missing values and they actually coexist in rows. Therefore, it is possible that the missing values are not random and depend on other characteristics of the customers. After checking with `income_type`, `education`, `gender`, `family_status` and `children` characteristics, we concluded that the missingness has no pattern.  

To fill up the missing values, we tried to find some diciding characteristics (factors) which affect the values of `total_income` and `days_employed` the most. If the means and medians of `total_income` (or `days_employed`) vary substantially across the categories in a characteristic, then the characteristic will be one of the deciding factor. 

In order to check the means and medians, some data transformations has been done. These include
- Dropping duplicates.
- Changing all the texts in `education` to lower case (this was done in the process of checking dependency of missing values).
- Replacing the value of '-1' in children with '0' since '-1' probably means no children.
- Replacing the value of '20' in `children` with '0' since this replacemnt has tiny or no effect on the large sample size of '0' category.
- Fixing negative values in `days_employed` by taking absolute values since this is a calculation error when doing subtraction.
- Fixing the unusual large values in `days_employed` by using the median of `days_employed` of customers aged 50 and above since 95% of these values come from retirees aged 50 and above.
- Replacing the value of '0' in `dob_years` with 'nan' since this artifacts is small in sample size and `dob_years` is not used in hypothesis testing.

After checking the means and medians of `total_income` and `days_employed` in various characteristics, we decided that
- `income_type`, `age_group` and `gender` were used to restore the missing values of `total_income`
- `age_group', 'family_status_id` and `income_type` were used to restore the missing values of `days_employed`
Since the distribution is skewed, an overall median of the three deciding factors was used to fill in the missing values.

At the end of the data preprocessing, `total_income` and `purpose` were categorised in order to test the hypotheses.

**Hypothesis testing**

This report aims to answer the following questions:
1. Is there a correlation between having children and paying back on time?
2. Is there a correlation between family status and paying back on time?
3. Is there a correlation between income level and paying back on time?
4. How does loan purpose affect the default rate?

Four relevant hypotheses have been tested:
1. Customers who have no children have a lower default rate compared to those have children.
2. Customers who are married have a lower default rate compared to those are unmarried
3. Customers with high income have the lowest default rate.
4. Customers who borrow a loan for education have the highest default rate.

After analysing the data, we concluded that

1. The customers without children have the lowest default rate in the `children` group. 

The first hypothesis is accepted.

2. The married customers have a lower default rate than the unmarried customers.  

The second hypothesis is accepted.

3. The customers of high income level have the lowest default rate.

The third hypothesis is accepted.

4. The customers who borrow loans for education have the second highest default rate.

The fourth hypothesis is rejected.

Other valuable findings:
1. The customers with 1 or 2 children tend to default on loan.
2. The widow / widower and the divorced categories are less likely to have loan default.
3. Customers with `family_status` of civil partnership and unmarried are more likely to default on loan.
4. The default rates of upper medium and lower medium income customers are slightly higher than that of the low income customers.
5. The customers who use the loan to buy car have the highest default rate in the `loan purpose` group.
6. The customers who use the loan for house/real estate purpose have the lowest default rate `loan purpose` group.

[Back to Contents](#back)